TO DO
- From gameweek 2 on, need to compare week n-1 and n to get some of the gameweek specific fpl data?!
- Develop code to handle no game / double game gameweeks (see, e.g. funtion get_was_home)

In [1]:
# number of latest gameweek
gameweek = 1
# check number of total players from fpl's site
total_players = 8654709

In [2]:
import pandas as pd
import numpy as np

import json
import requests
import sys

# allow more data columns to be shown than by default
pd.set_option('display.max_columns', 500)

In [3]:
sys.path.append('..')

In [4]:
import py_scripts.download_data as download_data

In [5]:
# dictionary for unifying the spelling of certain player names across fpl and fbref data
player_name_dict = {
    'Andreas Hoelgebaum Pereira':'Andreas Pereira',
    'Alisson Ramses Becker':'Alisson',
    'Bamidele Alli':'Dele Alli',
    'Benjamin Chilwell':'Ben Chilwell',
    'Bernardo Mota Veiga de Carvalho e Silva':'Bernardo Silva',
    'Bruno Guimaraes Rodriguez Moura':'Bruno Guimaraes',
    'Bruno Miguel Borges Fernandes':'Bruno Fernandes',
    'Cristiano Ronaldo dos Santos Aveiro':'Cristiano Ronaldo',
    'Daniel Castelo Podence':'Daniel Podence',
    'David Raya Martin':'David Raya',
    'Ederson Santana de Moraes':'Ederson',
    'Edward Nketiah':'Eddie Nketiah',
    'Emiliano Buendia Stati':'Emi Buendia',
    'Ezri Konsa Ngoyo':'Ezri Konsa',
    'Gabriel Fernando de Jesus':'Gabriel Jesus',
    'Gabriel Teodoro Martinelli Silva':'Martinelli',
    'Gabriel Magalhaes':'Gabriel Dos Santos',
    'Hee-Chan Hwang':'Hwang Hee-chan',
    'Son Heung-min':'Heung-Min Son',
    'Joao Filipe Iria Santos Moutinho':'Joao Moutinho',
    'Joao Pedro Cavaco Cancelo':'Joao Cancelo',
    'Joelinton Cassio Apolinario de Lira':'Joelinton',
    'Jose Diogo Dalot Teixeira':'Diogo Dalot',
    'Jose Malheiro de Sa':'Jose Sa',
    'Joseph Gomez':'Joe Gomez',
    'Joseph Willock':'Joe Willock',
    'Kiernan Dewsbury Hall':'Kiernan Dewsbury-Hall',
    'Konstantinos Tsimikas':'Kostas Tsimikas',
    'Lucas Rodrigues Moura da Silva':'Lucas Moura',
    'ukasz Fabianski':'Lukasz Fabianski',
    'Matthew Cash':'Matty Cash',
    'Pedro Lomba Neto':'Pedro Neto',
    'Philippe Coutinho Correia':'Philippe Coutinho',
    'Pierre Hjbjerg':'Pierre-Emile Hjbjerg',
    'Richarlison de Andrade':'Richarlison',
    'Ruben Diogo da Silva Neves':'Ruben Neves',
    'Ruben Santos Gato Alves Dias':'Ruben Dias',
    'Thiago Alcantara do Nascimento':'Thiago Alcantara',
    'Thiago Emiliano da Silva':'Thiago Silva',
    'Valentino Livramento':'Tino Livramento',
    'Vitalii Mykolenko':'Vitaliy Mykolenko',
    'Benjamin White':'Ben White',
    'Alex Nicolao Telles':'Alex Telles',
    'Bobby Reid':'Bobby De Cordova-Reid',
    'Bobby Decordova-Reid':'Bobby De Cordova-Reid',
    'Bruno Borges Fernandes':'Bruno Fernandes',
    'Darwin Nunez Ribeiro':'Darwin Nunez',
    'David De Gea Quintana':'David De Gea',
    'Diogo Dalot Teixeira':'Diogo Dalot',
    'Emiliano Martinez Romero':'Emiliano Martinez',
    'Fabio Freitas Gouveia Carvalho':'Fabio Carvalho',
    'Francisco Machado Mota de Castro Trincao':'Francisco Trincao',
    'Gabriel Martinelli Silva':'Martinelli',
    'Gabriel dos Santos Magalhaes':'Gabriel Dos Santos',
    'Jefferson Lerma Solis':'Jefferson Lerma',
    'Joao Palhinha Goncalves':'Joao Palhinha',
    'Marc Cucurella Saseta':'Marc Cucurella',
    'Moises Caicedo Corozo':'Moises Caicedo',
    'Nuno Varela Tavares':'Nuno Tavares',
    'Nelson Cabral Semedo':'Nelson Semedo',
    'Oriol Romeu Vidal':'Oriol Romeu',
    'Pablo Fornals Malla':'Pablo Fornals',
    'Pelenda Joshua Dasilva':'Josh Dasilva',
    'Rasmus Nissen':'Rasmus Kristensen',
    'Rayan Ait Nouri':'Rayan Ait-Nouri',
    'Ricardo Domingos Barbosa Pereira':'Ricardo Pereira',
    'Rodrigo Hernandez':'Rodri',
    'Rodrigo Moreno':'Rodrigo',
    'Ruben Gato Alves Dias':'Ruben Dias',
    'Ruben Goncalo Silva Nascimento Vinagre':'Ruben Vinagre',
    'Ruben Nascimento Vinagre':'Ruben Vinagre',
    'Ruben da Silva Neves':'Ruben Neves',
}

# Fetch FPL data

In [6]:
# for some reason, fpl team numbers are otherwise alphabetic, except Leeds comes after Leicester...
teams = ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
        'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leicester City', 'Leeds United',
         'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd',
       'Nottingham Forest', 'Southampton', 'Tottenham', 'West Ham','Wolves']

In [7]:
# fetch FPL data online
fpl_orig_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_data = pd.DataFrame(fpl_orig_data['elements'])
fpl_data.set_index('id',inplace=True)
fpl_data['season'] = '2022-23'
fpl_data['name'] = fpl_data['first_name'] + ' ' + fpl_data['second_name']
# normalize spelling of names
fpl_data['name'] = fpl_data['name'].str.normalize('NFD').str.encode('ascii','ignore').str.decode('ascii')
fpl_data['name'] = fpl_data['name'].map(player_name_dict).fillna(fpl_data['name'])
position_dict={1:'GK', 2:'DEF', 3:'MID', 4:'FWD'}
fpl_data['position'] = fpl_data['element_type'].map(position_dict)
# change team number to team name
fpl_data['team'] = [teams[i-1] for i in fpl_data['team']]
fpl_data['GW'] = gameweek
fpl_data['value'] = fpl_data['now_cost']/10.0
fpl_data['selected'] = (fpl_data['selected_by_percent'].astype(float) / 100) * total_players
fpl_data['transfers_balance'] = fpl_data['transfers_in'] - fpl_data['transfers_out']
fpl_data

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,season,name,position,GW,value,selected,transfers_balance
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,58822,0,0,0,0,0,2,2.4,2.4,0,Cédric,0.0,False,,None,45,58822.jpg,0.0,Alves Soares,0.3,False,None,a,Arsenal,3,0,1180,1180,6007,6007,0.0,0.0,Cédric,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,345,148,341,145,279,113,361,148,4.0,,3.0,,NaN,,2022-23,Cedric Alves Soares,DEF,1,4.5,25964.127,-4827
3,NaN,NaN,84450,0,0,0,0,0,3,2.1,2.1,2,Granit,2.0,False,,None,50,84450.jpg,2.0,Xhaka,0.7,False,None,a,Arsenal,3,2,8050,8050,8788,8788,0.4,0.4,Xhaka,90,0,0,1,0,0,0,0,1,0,0,0,12,16.6,15.0,2.0,3.4,98,32,81,52,167,90,123,65,NaN,,NaN,,NaN,,2022-23,Granit Xhaka,MID,1,5.0,60582.963,-738
4,NaN,NaN,153256,0,0,0,0,0,3,1.5,1.5,0,Mohamed,0.0,False,,None,45,153256.jpg,0.0,Elneny,1.0,False,None,a,Arsenal,3,0,5353,5353,17006,17006,0.0,0.0,Elneny,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,380,130,377,140,321,115,396,139,NaN,,NaN,,NaN,,2022-23,Mohamed Elneny,MID,1,4.5,86547.090,-11653
5,NaN,NaN,156074,0,0,0,0,0,2,2.4,2.4,0,Rob,0.0,False,,None,45,156074.jpg,0.0,Holding,0.2,False,None,a,Arsenal,3,0,523,523,3328,3328,0.0,0.0,Holding,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,383,168,382,165,326,138,399,168,NaN,,NaN,,NaN,,2022-23,Rob Holding,DEF,1,4.5,17309.418,-2805
6,NaN,NaN,167199,0,0,0,0,0,3,2.1,2.1,3,Thomas,3.0,False,,None,50,167199.jpg,3.0,Partey,0.7,False,None,a,Arsenal,3,3,11320,11320,7903,7903,0.6,0.6,Partey,90,0,0,1,0,0,0,0,0,0,0,0,17,22.6,12.9,10.0,4.6,61,23,95,58,96,57,85,48,NaN,,4.0,,NaN,,2022-23,Thomas Partey,MID,1,5.0,60582.963,3417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,NaN,NaN,490721,0,0,0,0,0,2,1.5,1.5,0,Hugo,0.0,False,,None,40,490721.jpg,0.0,Bueno López,0.7,False,None,a,Wolves,39,0,6739,6739,3870,3870,0.0,0.0,Bueno,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,328,134,323,131,257,96,344,134,NaN,,NaN,,NaN,,2022-23,Hugo Bueno Lopez,DEF,1,4.0,60582.963,2869
566,NaN,NaN,240299,0,0,0,0,0,3,1.5,1.5,0,Joseph,0.0,False,,None,45,240299.jpg,0.0,Hodge,0.0,False,None,a,Wolves,39,0,701,701,169,169,0.0,0.0,Hodge,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,422,150,427,160,387,136,438,159,NaN,,NaN,,NaN,,2022-23,Joseph Hodge,MID,1,4.5,0.000,532
567,NaN,NaN,198842,0,0,0,0,0,3,1.5,1.5,0,Connor,0.0,False,,None,45,198842.jpg,0.0,Ronan,0.0,False,None,a,Wolves,39,0,1588,1588,394,394,0.0,0.0,Ronan,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,396,138,396,148,347,124,412,147,NaN,,NaN,,NaN,,2022-23,Connor Ronan,MID,1,4.5,0.000,1194


In [8]:
# get FPL fixtures data
fpl_fixtures_data = json.loads(requests.get('https://fantasy.premierleague.com/api/fixtures/').text)
fpl_fixtures = pd.DataFrame(fpl_fixtures_data)
fpl_fixtures['home_team'] = [teams[i] for i in fpl_fixtures['team_h']-1]
fpl_fixtures['away_team'] = [teams[i] for i in fpl_fixtures['team_a']-1]
fpl_fixtures

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,home_team,away_team
0,2292810,1,True,True,1,2022-08-05T19:00:00Z,90,False,True,1,2.0,7,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,2,74911,Crystal Palace,Arsenal
1,2292813,1,True,True,4,2022-08-06T11:30:00Z,90,False,True,12,2.0,9,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,74914,Fulham,Liverpool
2,2292811,1,True,True,2,2022-08-06T14:00:00Z,90,False,True,2,0.0,3,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,2,74912,Bournemouth,Aston Villa
3,2292814,1,True,True,5,2022-08-06T14:00:00Z,90,False,True,20,1.0,11,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,74915,Leeds United,Wolves
4,2292816,1,True,True,7,2022-08-06T14:00:00Z,90,False,True,16,0.0,15,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,3,74917,Newcastle Utd,Nottingham Forest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2293185,38,False,False,376,2023-05-28T15:00:00Z,0,False,False,3,NaN,8,NaN,[],2,2,75286,Everton,Bournemouth
376,2293186,38,False,False,377,2023-05-28T15:00:00Z,0,False,False,18,NaN,11,NaN,[],3,2,75287,Leeds United,Tottenham
377,2293187,38,False,False,378,2023-05-28T15:00:00Z,0,False,False,19,NaN,10,NaN,[],3,4,75288,Leicester City,West Ham
378,2293188,38,False,False,379,2023-05-28T15:00:00Z,0,False,False,9,NaN,14,NaN,[],2,4,75289,Manchester Utd,Fulham


In [9]:
def get_opponent(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    # my_team is the home team
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team)].shape[0]==1:
        opponent_team = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team), \
                                         'away_team']
    # my_team is the away team
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team)].shape[0]==1:
        opponent_team = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team), \
                                         'home_team']
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & ( (fpl_fixtures['home_team']==my_team) | 
                      (fpl_fixtures['away_team']==my_team) )].shape[0]!=1:
        print(f'Number of games not equal to one for ix {my_row.index}')

    return opponent_team.values[0]

In [10]:
def get_kickoff_time(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    kickoff_time = fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & ( (fpl_fixtures['home_team']==my_team) | 
                      (fpl_fixtures['away_team']==my_team) ), 'kickoff_time']
    if kickoff_time.shape[0]==1:
        return kickoff_time.values[0]
    else:
        print(f'Kickoff_time shape not one for index {my_row.index}.')

In [11]:
def get_was_home(my_row, fpl_fixtures, gameweek):
    my_team = my_row['team']
    if fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['home_team']==my_team)].shape[0]==1:
        return 1
    elif fpl_fixtures.loc[(fpl_fixtures['event']==gameweek) & (fpl_fixtures['away_team']==my_team)].shape[0]==1:
        return 0
    else:
        print(f'Error in get_was_home for index {my_row.index}.')

In [12]:
fpl_data['opp_team_name'] = fpl_data.apply(lambda x: get_opponent(x, fpl_fixtures, gameweek), axis=1)
fpl_data['kickoff_time'] = fpl_data.apply(lambda x: get_kickoff_time(x, fpl_fixtures, gameweek), axis=1)
fpl_data['was_home'] = fpl_data.apply(lambda x: get_was_home(x, fpl_fixtures, gameweek), axis=1)
fpl_data

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,season,name,position,GW,value,selected,transfers_balance,opp_team_name,kickoff_time,was_home
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,58822,0,0,0,0,0,2,2.4,2.4,0,Cédric,0.0,False,,None,45,58822.jpg,0.0,Alves Soares,0.3,False,None,a,Arsenal,3,0,1180,1180,6007,6007,0.0,0.0,Cédric,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,345,148,341,145,279,113,361,148,4.0,,3.0,,NaN,,2022-23,Cedric Alves Soares,DEF,1,4.5,25964.127,-4827,Crystal Palace,2022-08-05T19:00:00Z,0
3,NaN,NaN,84450,0,0,0,0,0,3,2.1,2.1,2,Granit,2.0,False,,None,50,84450.jpg,2.0,Xhaka,0.7,False,None,a,Arsenal,3,2,8050,8050,8788,8788,0.4,0.4,Xhaka,90,0,0,1,0,0,0,0,1,0,0,0,12,16.6,15.0,2.0,3.4,98,32,81,52,167,90,123,65,NaN,,NaN,,NaN,,2022-23,Granit Xhaka,MID,1,5.0,60582.963,-738,Crystal Palace,2022-08-05T19:00:00Z,0
4,NaN,NaN,153256,0,0,0,0,0,3,1.5,1.5,0,Mohamed,0.0,False,,None,45,153256.jpg,0.0,Elneny,1.0,False,None,a,Arsenal,3,0,5353,5353,17006,17006,0.0,0.0,Elneny,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,380,130,377,140,321,115,396,139,NaN,,NaN,,NaN,,2022-23,Mohamed Elneny,MID,1,4.5,86547.090,-11653,Crystal Palace,2022-08-05T19:00:00Z,0
5,NaN,NaN,156074,0,0,0,0,0,2,2.4,2.4,0,Rob,0.0,False,,None,45,156074.jpg,0.0,Holding,0.2,False,None,a,Arsenal,3,0,523,523,3328,3328,0.0,0.0,Holding,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,383,168,382,165,326,138,399,168,NaN,,NaN,,NaN,,2022-23,Rob Holding,DEF,1,4.5,17309.418,-2805,Crystal Palace,2022-08-05T19:00:00Z,0
6,NaN,NaN,167199,0,0,0,0,0,3,2.1,2.1,3,Thomas,3.0,False,,None,50,167199.jpg,3.0,Partey,0.7,False,None,a,Arsenal,3,3,11320,11320,7903,7903,0.6,0.6,Partey,90,0,0,1,0,0,0,0,0,0,0,0,17,22.6,12.9,10.0,4.6,61,23,95,58,96,57,85,48,NaN,,4.0,,NaN,,2022-23,Thomas Partey,MID,1,5.0,60582.963,3417,Crystal Palace,2022-08-05T19:00:00Z,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558,NaN,NaN,490721,0,0,0,0,0,2,1.5,1.5,0,Hugo,0.0,False,,None,40,490721.jpg,0.0,Bueno López,0.7,False,None,a,Wolves,39,0,6739,6739,3870,3870,0.0,0.0,Bueno,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,328,134,323,131,257,96,344,134,NaN,,NaN,,NaN,,2022-23,Hugo Bueno Lopez,DEF,1,4.0,60582.963,2869,Leeds United,2022-08-06T14:00:00Z,0
566,NaN,NaN,240299,0,0,0,0,0,3,1.5,1.5,0,Joseph,0.0,False,,None,45,240299.jpg,0.0,Hodge,0.0,False,None,a,Wolves,39,0,701,701,169,169,0.0,0.0,Hodge,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,422,150,427,160,387,136,438,159,NaN,,NaN,,NaN,,2022-23,Joseph Hodge,MID,1,4.5,0.000,532,Leeds United,2022-08-06T14:00:00Z,0
567,NaN,NaN,198842,0,0,0,0,0,3,1.5,1.5,0,Connor,0.0,False,,None,45,198842.jpg,0.0,Ronan,0.0,False,None,a,Wolves,39,0,1588,1588,394,394,0.0,0.0,Ronan,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,396,138,396,148,347,124,412,147,NaN,,NaN,,NaN,,2022-23,Connor Ronan,MID,1,4.5,0.000,1194,Leeds United,2022-08-06T14:00:00Z,0


In [13]:
fpl_data.to_csv(f'../data/fpl_data/fpl_week_{gameweek}.csv')
fpl_fixtures.to_csv('../data/fpl_fixtures.csv')

In [14]:
# create a csv that contains each week's fpl data in one table
if gameweek==1:
    fpl_data['total_points'] = fpl_data['event_points']
    fpl_data[fpl_data['minutes']>0].to_csv('../data/fpl_data.csv')

# Fetch fbref data

In [15]:
url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
links = download_data.get_match_report_links(url)
links

['https://fbref.com/en/matches/e62f6e78/Crystal-Palace-Arsenal-August-5-2022-Premier-League',
 'https://fbref.com/en/matches/6713c1dc/Fulham-Liverpool-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/877e3193/Bournemouth-Aston-Villa-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/09d8a999/Tottenham-Hotspur-Southampton-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/82702941/Leeds-United-Wolverhampton-Wanderers-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/1ac96eb4/Newcastle-United-Nottingham-Forest-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/3a917cee/Everton-Chelsea-August-6-2022-Premier-League',
 'https://fbref.com/en/matches/3249ba27/Leicester-City-Brentford-August-7-2022-Premier-League',
 'https://fbref.com/en/matches/8251694e/Manchester-United-Brighton-and-Hove-Albion-August-7-2022-Premier-League',
 'https://fbref.com/en/matches/ece62baf/West-Ham-United-Manchester-City-August-7-2022-Premier-League']

In [16]:
fbref_data = download_data.download_match_report_data(links)
# normalize spelling of player names
fbref_data[('General','Player')] = fbref_data[('General','Player')].str.normalize('NFD').str.encode('ascii','ignore').str.decode('ascii')
# map certain player names such that they are the same for both fpl and fbref data
fbref_data[('General','Player')] = fbref_data[('General','Player')].map(player_name_dict).fillna(fbref_data[('General','Player')])
fbref_data

100%|███████████████████████████████████████████| 10/10 [00:37<00:00,  3.75s/it]


General                                   Performance      \
                   Player     #   Nation    Pos     Age Min         Gls Ast   
0         Odsonne Edouard  22.0   fr FRA     FW  24-201  57           0   0   
1    Jean-Philippe Mateta  14.0   fr FRA     FW  25-038  33           0   0   
2           Wilfried Zaha  11.0   ci CIV     LW  29-268  90           0   0   
3             Jordan Ayew   9.0   gh GHA  RW,AM  30-328  90           0   0   
4            Eberechi Eze  10.0  eng ENG     AM  24-037  85           0   0   
..                    ...   ...      ...    ...     ...  ..         ...  ..   
292          Joao Cancelo   7.0   pt POR     LB  28-072  90           0   0   
293            Nathan Ake   6.0   nl NED     CB  27-170  90           0   0   
294            Ruben Dias   3.0   pt POR     CB  25-085  90           0   0   
295           Kyle Walker   2.0  eng ENG     RB  32-071  90           0   0   
296               Ederson  31.0   br BRA     GK  28-355  90           0   0   

                                                           Expected            \
    PK PKatt Sh SoT CrdY CrdR Touches Press Tkl Int Blocks       xG npxG   xA   
0    0     0  3   1    0    0      21    19   1   0      3      0.3  0.3  0.0   
1    0     0  1   0    0    0      12     6   0   0      0      0.1  0.1  0.0   
2    0     0  1   0    0    0      59    19   2   0      0      0.0  0.0  0.4   
3    0     0  1   0    0    0      43    28   2   1      0      0.1  0.1  0.1   
4    0     0  1   1    0    0      54    20   1   1      2      0.4  0.4  0.0   
..  ..   ... ..  ..  ...  ...     ...   ...  ..  ..    ...      ...  ...  ...   
292  0     0  1   0    1    0     124     5   1   2      1      0.0  0.0  0.1   
293  0     0  1   0    0    0     113     5   1   1      3      0.1  0.1  0.0   
294  0     0  0   0    0    0     105     5   0   0      0      0.0  0.0  0.0   
295  0     0  0   0    0    0     100     4   0   0      1      0.0  0.0  0.0   
296  0     0  0   0    0    0      42     0   0   0      0      0.0  0.0  0.0   

    SCA     Passes                 Carries      Dribbles              General  \
    SCA GCA    Cmp  Att  Cmp% Prog Carries Prog     Succ Att            Squad   
0     1   0      4    7  57.1    0       9    2        0   2   Crystal Palace   
1     1   0      9   11  81.8    0       7    1        0   0   Crystal Palace   
2     3   0     29   40  72.5    3      54   10        2   6   Crystal Palace   
3     6   0     26   33  78.8    1      42    5        5   7   Crystal Palace   
4     1   0     32   42  76.2    1      35    3        0   2   Crystal Palace   
..   ..  ..    ...  ...   ...  ...     ...  ...      ...  ..              ...   
292   4   0    113  120  94.2   10     105   13        3   4  Manchester City   
293   0   0     99  104  95.2    1      93    3        0   0  Manchester City   
294   0   0     96   98  98.0    5      89    6        0   0  Manchester City   
295   1   0     87   96  90.6    3      79   12        0   0  Manchester City   
296   0   0     38   42  90.5    0      31    0        0   0  Manchester City   

                       Total                            Short             \
    HomeTeam  Opponent   Cmp  Att  Cmp% TotDist PrgDist   Cmp Att   Cmp%   
0       True   Arsenal     4    7  57.1      49       4     3   4   75.0   
1       True   Arsenal     9   11  81.8     125       0     5   6   83.3   
2       True   Arsenal    29   40  72.5     422     107    17  22   77.3   
3       True   Arsenal    26   33  78.8     469      40    13  16   81.3   
4       True   Arsenal    32   42  76.2     662     135    12  15   80.0   
..       ...       ...   ...  ...   ...     ...     ...   ...  ..    ...   
292    False  West Ham   113  120  94.2    1968     516    60  61   98.4   
293    False  West Ham    99  104  95.2    1751     688    47  49   95.9   
294    False  West Ham    96   98  98.0    1876     674    36  36  100.0   
295    False  West Ham    87   96  90.6    134

In [17]:
fbref_data[('General','Player')] = fbref_data[('General','Player')].map(player_name_dict).fillna(fbref_data[('General','Player')])
fbref_data

General                                   Performance      \
                   Player     #   Nation    Pos     Age Min         Gls Ast   
0         Odsonne Edouard  22.0   fr FRA     FW  24-201  57           0   0   
1    Jean-Philippe Mateta  14.0   fr FRA     FW  25-038  33           0   0   
2           Wilfried Zaha  11.0   ci CIV     LW  29-268  90           0   0   
3             Jordan Ayew   9.0   gh GHA  RW,AM  30-328  90           0   0   
4            Eberechi Eze  10.0  eng ENG     AM  24-037  85           0   0   
..                    ...   ...      ...    ...     ...  ..         ...  ..   
292          Joao Cancelo   7.0   pt POR     LB  28-072  90           0   0   
293            Nathan Ake   6.0   nl NED     CB  27-170  90           0   0   
294            Ruben Dias   3.0   pt POR     CB  25-085  90           0   0   
295           Kyle Walker   2.0  eng ENG     RB  32-071  90           0   0   
296               Ederson  31.0   br BRA     GK  28-355  90           0   0   

                                                           Expected            \
    PK PKatt Sh SoT CrdY CrdR Touches Press Tkl Int Blocks       xG npxG   xA   
0    0     0  3   1    0    0      21    19   1   0      3      0.3  0.3  0.0   
1    0     0  1   0    0    0      12     6   0   0      0      0.1  0.1  0.0   
2    0     0  1   0    0    0      59    19   2   0      0      0.0  0.0  0.4   
3    0     0  1   0    0    0      43    28   2   1      0      0.1  0.1  0.1   
4    0     0  1   1    0    0      54    20   1   1      2      0.4  0.4  0.0   
..  ..   ... ..  ..  ...  ...     ...   ...  ..  ..    ...      ...  ...  ...   
292  0     0  1   0    1    0     124     5   1   2      1      0.0  0.0  0.1   
293  0     0  1   0    0    0     113     5   1   1      3      0.1  0.1  0.0   
294  0     0  0   0    0    0     105     5   0   0      0      0.0  0.0  0.0   
295  0     0  0   0    0    0     100     4   0   0      1      0.0  0.0  0.0   
296  0     0  0   0    0    0      42     0   0   0      0      0.0  0.0  0.0   

    SCA     Passes                 Carries      Dribbles              General  \
    SCA GCA    Cmp  Att  Cmp% Prog Carries Prog     Succ Att            Squad   
0     1   0      4    7  57.1    0       9    2        0   2   Crystal Palace   
1     1   0      9   11  81.8    0       7    1        0   0   Crystal Palace   
2     3   0     29   40  72.5    3      54   10        2   6   Crystal Palace   
3     6   0     26   33  78.8    1      42    5        5   7   Crystal Palace   
4     1   0     32   42  76.2    1      35    3        0   2   Crystal Palace   
..   ..  ..    ...  ...   ...  ...     ...  ...      ...  ..              ...   
292   4   0    113  120  94.2   10     105   13        3   4  Manchester City   
293   0   0     99  104  95.2    1      93    3        0   0  Manchester City   
294   0   0     96   98  98.0    5      89    6        0   0  Manchester City   
295   1   0     87   96  90.6    3      79   12        0   0  Manchester City   
296   0   0     38   42  90.5    0      31    0        0   0  Manchester City   

                       Total                            Short             \
    HomeTeam  Opponent   Cmp  Att  Cmp% TotDist PrgDist   Cmp Att   Cmp%   
0       True   Arsenal     4    7  57.1      49       4     3   4   75.0   
1       True   Arsenal     9   11  81.8     125       0     5   6   83.3   
2       True   Arsenal    29   40  72.5     422     107    17  22   77.3   
3       True   Arsenal    26   33  78.8     469      40    13  16   81.3   
4       True   Arsenal    32   42  76.2     662     135    12  15   80.0   
..       ...       ...   ...  ...   ...     ...     ...   ...  ..    ...   
292    False  West Ham   113  120  94.2    1968     516    60  61   98.4   
293    False  West Ham    99  104  95.2    1751     688    47  49   95.9   
294    False  West Ham    96   98  98.0    1876     674    36  36  100.0   
295    False  West Ham    87   96  90.6    134

In [18]:
# save links that have been downloaded already so that in the future one can only fetch data from new games
with open('../data/fbref_fixture_links.txt', 'w') as f:
    f.write('\n'.join(links))

In [19]:
# how to read the links
#with open('../../data/fbref_fixture_links.txt') as f:
#    new_links = f.read().splitlines()
#new_links

In [20]:
fbref_data.to_csv(f'../data/fbref_data.csv')

## 22-23 Match results

In [21]:
url = 'https://fbref.com/en/comps/9/schedule/Premier-League-Scores-and-Fixtures'
data = pd.read_html(url)
results = data[0].dropna(axis=0, how='all').copy()
results

,Wk,Day,Date,Time,Home,xG,Score,xG.1,Away,Attendance,Venue,Referee,Match Report,Notes
0,1.0,Fri,2022-08-05,20:00,Crystal Palace,0.9,0–2,1.0,Arsenal,25286.0,Selhurst Park,Anthony Taylor,Match Report,NaN
1,1.0,Sat,2022-08-06,12:30,Fulham,1.4,2–2,1.9,Liverpool,22207.0,Craven Cottage,Andy Madley,Match Report,NaN
2,1.0,Sat,2022-08-06,15:00,Bournemouth,0.5,2–0,0.6,Aston Villa,11013.0,Vitality Stadium,Peter Bankes,Match Report,NaN
3,1.0,Sat,2022-08-06,15:00,Tottenham,1.5,4–1,0.5,Southampton,61732.0,Tottenham Hotspur Stadium,Andre Marriner,Match Report,NaN
4,1.0,Sat,2022-08-06,15:00,Leeds United,0.6,2–1,1.3,Wolves,36347.0,Elland Road,Robert Jones,Match Report,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,38.0,Sun,2023-05-28,15:00,Everton,NaN,NaN,NaN,Bournemouth,NaN,Goodison Park,NaN,Head-to-Head,NaN
413,38.0,Sun,2023-05-28,15:00,Aston Villa,NaN,NaN,NaN,Brighton,NaN,Villa Park,NaN,Head-to-Head,NaN
414,38.0,Sun,2023-05-28,15:00,Leeds United,NaN,NaN,NaN,Tottenham,NaN,Elland Road,NaN,Head-to-Head,NaN
415,38.0,Sun,2023-05-28,15:00,Chelsea,NaN,NaN,NaN,Newcastle Utd,NaN,Stamford Bridge,NaN,Head-to-Head,NaN


In [22]:
# fix Nott Forest team name
my_dict = {"Nott'ham Forest":'Nottingham Forest'}
results['Home'] = results['Home'].map(my_dict).fillna(results['Home'])
results['Away'] = results['Away'].map(my_dict).fillna(results['Away'])

In [23]:
results.to_csv('../data/PL_results_season22_23.csv')